In [1]:
import string

In [85]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Lambda
from keras.layers import Conv1D, MaxPooling1D, Flatten, Concatenate, LSTM, Conv2D, MaxPool2D
from keras.models import Model
from keras.initializers import Constant
from keras import regularizers
import matplotlib.pyplot as plt
from keras import backend as TF

import numpy as np
import string
import pandas as pd
import nltk
import keras

from sklearn import random_projection
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords

from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, CuDNNLSTM, Bidirectional, Reshape, CuDNNGRU
from keras.optimizers import SGD, Adam, RMSprop
from keras import metrics
import tensorflow as tf
from attention_with_context import AttentionWithContext

In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()



['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
z = string.punctuation
set(z)
punct = [letter for letter in string.punctuation]
stop_words = set(stopwords.words('english') + punct)



In [5]:
def tokenize(text):
    '''
    :param text: a doc with multiple sentences, type: str
    return a word list, type: list
    https://textminingonline.com/dive-into-nltk-part-ii-sentence-tokenize-and-word-tokenize
    e.g.
    Input: 'It is a nice day. I am happy.'
    Output: ['it', 'is', 'a', 'nice', 'day', 'i', 'am', 'happy']
    '''
    tokens = []
    for word in nltk.word_tokenize(text):
        word = word.lower()
        if word not in stop_words and not word.isnumeric():
            tokens.append(word)
    return tokens

In [6]:
def get_sequence(data, seq_length, vocab_dict):
    '''
    :param data: a list of words, type: list
    :param seq_length: the length of sequences,, type: int
    :param vocab_dict: a dict from words to indices, type: dict
    return a dense sequence matrix whose elements are indices of words,
    '''
    data_matrix = np.zeros((len(data), seq_length), dtype=int)
    for i, doc in enumerate(data):
        for j, word in enumerate(doc):
            # YOUR CODE HERE
            if j == seq_length:
                break
            word_idx = vocab_dict.get(word, 1) # 1 means the unknown word
            data_matrix[i, j] = word_idx
    return data_matrix

In [77]:
def get_embeddings():
    embeddings_index = dict();
    with open('data/glove.6B.300d.txt') as f:
        for line in f:
            values = line.split();
            word = values[0];
            coefs = np.asarray(values[1:], dtype='float32');
            embeddings_index[word] = coefs
    return embeddings_index

def read_data(file_name, input_length, vocab=None):
    """
    https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
    """
    df = pd.read_csv(file_name)
    df['words'] = df['text'].apply(tokenize)
    
    if vocab is None:
        vocab = set()
        for i in range(len(df)):
            for word in df.iloc[i]['words']:
                vocab.add(word)
    vocab_dict = dict()
    vocab_dict['<pad>'] = 0 # 0 means the padding signal
    vocab_dict['<unk>'] = 1 # 1 means the unknown word
    vocab_size = 2
    for v in vocab:
        vocab_dict[v] = vocab_size
        vocab_size += 1

    data_matrix = get_sequence(df['words'], input_length, vocab_dict)
    stars = df['stars'].apply(int) - 1
    return df['review_id'], stars, data_matrix, vocab, vocab_dict
# ----------------- End of Helper Functions-----------------


def load_data(input_length):
    # Load training data and vocab
    train_id_list, train_data_label, train_data_matrix, vocab, vocab_dict = read_data("data/train.csv", input_length)
    K = max(train_data_label)+1  # labels begin with 0

    # Load valid data
    valid_id_list, valid_data_label, valid_data_matrix, vocab, vocab_dict = read_data("data/valid.csv", input_length, vocab=vocab)

    # Load testing data
    test_id_list, _, test_data_matrix, _, _= read_data("data/test.csv", input_length, vocab=vocab)

    print("Vocabulary Size:", len(vocab))
    print("Training Set Size:", len(train_id_list))
    print("Validation Set Size:", len(valid_id_list))
    print("Test Set Size:", len(test_id_list))
    print("Training Set Shape:", train_data_matrix.shape)
    print("Validation Set Shape:", valid_data_matrix.shape)
    print("Testing Set Shape:", test_data_matrix.shape)

    # Converts a class vector to binary class matrix.
    # https://keras.io/utils/#to_categorical
    train_data_label = keras.utils.to_categorical(train_data_label, num_classes=K)
    valid_data_label = keras.utils.to_categorical(valid_data_label, num_classes=K)
    return train_id_list, train_data_matrix, train_data_label, \
        valid_id_list, valid_data_matrix, valid_data_label, \
        test_id_list, test_data_matrix, None, vocab, vocab_dict


In [79]:
input_length = 200
embedding_size = 300 #300
hidden_size = 100 #200 # 64
batch_size = 128
dropout_rate = 0.5
embedd_dropout = 0.5
learning_rate = 0.001
total_epoch = 30
word_cutoff = 5 


In [71]:
train_id_list, train_data_matrix, train_data_label, \
        valid_id_list, valid_data_matrix, valid_data_label, \
        test_id_list, test_data_matrix, _, vocab, vocab_dict = load_data(input_length)

Vocabulary Size: 114544
Training Set Size: 100000
Validation Set Size: 10000
Test Set Size: 10000
Training Set Shape: (100000, 200)
Validation Set Shape: (10000, 200)
Testing Set Shape: (10000, 200)


In [80]:
vocab_dict
len(vocab)
N = train_data_matrix.shape[0]
K = train_data_label.shape[1]
EMBEDDING_DIM = 300

input_size = len(vocab) + 2
output_size = K

kernel_sizes = [3, 4, 5]
padding = 'valid'
activation = 'relu'
strides = 1
pool_size = 2
filters = 64

In [81]:
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    return output_attention_mul



In [82]:
embeddings_index = get_embeddings() 
embedding_matrix = np.zeros((input_size, EMBEDDING_DIM))
absent_words = 0 
for word, i in vocab_dict.items():
#     if i > MAX_NUM_WORDS:
#         continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else: 
        absent_words +=1 
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
print ("total absent words are: {}".format(absent_words))
embedding_layer = Embedding(input_size,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=input_length,
                            trainable=False)


total absent words are: 57149


In [ ]:
sequence_input = Input(shape=(input_length,), dtype='float32')
# embed_seq = Embedding(input_size, embedding_size, input_length=input_length)(sequence_input)
# embed_drop = Dropout(embedd_dropout)(embed_seq)
embed_seq = embedding_layer(sequence_input)
# conv_blocks = []
# for kernel_size in kernel_sizes:
#     conv = Conv1D(filters,
#                  kernel_size,
#                  padding='valid',
#                  activation='relu',
#                  strides=1)(embed_seq)
#     maxpooling = MaxPooling1D(pool_size=pool_size)(conv)
# #     conv = Conv2D(filters=filters, kernel_size=(kernel_size, embedding_size), padding=padding, activation=activation, strides=(strides, strides))(conv_input)
# #     maxpooling = MaxPool2D(pool_size=((input_length-kernel_size)//strides+1, 1))(conv)
#     conv_blocks.append(maxpooling)
# c = Concatenate()(conv_blocks) if len(kernel_sizes) > 1 else conv_blocks[0]
# c_d = Dropout(dropout_rate)(c)
#lstm_output, lstm_state, lstm_cstate = \
#Bidirectional(CuDNNLSTM(units=hidden_size, return_state=True), merge_mode='concat')(c_d)
lstm_o = CuDNNLSTM(units=embedding_size, return_sequences=True)(embed_seq)
lstm_expanded = Lambda(lambda x: TF.expand_dims(x, -1))(lstm_o)
    # CNN layers
conv_blocks = []
for kernel_size in kernel_sizes:
        # YOUR CODE HERE
    conv = Conv2D(filters=filters, kernel_size=(kernel_size, \
         embedding_size) ,padding=padding, activation=activation, \
         strides=(strides, strides))(lstm_expanded)
    maxpooling = MaxPool2D(pool_size=( \
        (input_length-kernel_size)//strides+1, 1))(conv)
    faltten = Flatten()(maxpooling)
    conv_blocks.append(faltten)
c = Concatenate()(conv_blocks) if len(kernel_sizes) > 1 else conv_blocks[0]
# c_r = Lambda(lambda x: TF.reshape(x, [-1, filters]))(c)
c_d = Dropout(dropout_rate)(c)
preds = Dense(K, activation='softmax')(c_d)
model = Model(sequence_input, preds)
     # SGD optimizer with momentum
optimizer = Adam(lr=learning_rate, decay=1e-)
     # compile model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    # training
model.fit(train_data_matrix, train_data_label, epochs=total_epoch, batch_size=batch_size, validation_data=(valid_data_matrix, valid_data_label))
     # testing
train_score = model.evaluate(train_data_matrix, train_data_label, batch_size=batch_size)
print('Training Loss: {}\n Training Accuracy: {}\n'.format(train_score[0], train_score[1]))
valid_score = model.evaluate(valid_data_matrix, valid_data_label, batch_size=batch_size)
print('Validation Loss: {}\n Validation Accuracy: {}\n'.format(valid_score[0], valid_score[1]))

    # predicting
test_pre = model.predict(test_data_matrix, batch_size=batch_size).argmax(axis=-1) + 1
sub_df = pd.DataFrame()

Train on 100000 samples, validate on 10000 samples
Epoch 1/30
100000/100000 [==============================] - 101s 1ms/step - loss: 0.9694 - acc: 0.6032 - val_loss: 0.9485 - val_acc: 0.6099
Epoch 2/30
100000/100000 [==============================] - 100s 996us/step - loss: 0.8510 - acc: 0.6517 - val_loss: 0.8787 - val_acc: 0.6454
Epoch 3/30
100000/100000 [==============================] - 100s 1ms/step - loss: 0.7865 - acc: 0.6703 - val_loss: 0.7753 - val_acc: 0.6755
Epoch 4/30
100000/100000 [==============================] - 99s 993us/step - loss: 0.7430 - acc: 0.6902 - val_loss: 0.8309 - val_acc: 0.6639
Epoch 5/30
100000/100000 [==============================] - 99s 986us/step - loss: 0.7006 - acc: 0.7049 - val_loss: 0.8571 - val_acc: 0.6564
Epoch 6/30
100000/100000 [==============================] - 98s 984us/step - loss: 0.6578 - acc: 0.7236 - val_loss: 0.8292 - val_acc: 0.6698
Epoch 7/30
100000/100000 [==============================] - 101s 1ms/step - loss: 0.6111 - acc: 0.7412 -

# sequence_input = Input(shape=(input_length,), dtype='float32')
embedded_sequences = Embedding(input_dim=input_size, output_dim=embedding_size, 
                                            input_length=input_length)(sequence_input)
embedd_dropout = Dropout(dropout_rate)(embedded_sequences)
word_lstm = Bidirectional(CuDNNGRU(input_size, return_sequences=True))(embedd_dropout)
word_dense = TimeDistributed(Dense(hidden, kernel_regularizer=l2_reg))(word_lstm)
word_att = AttentionWithContext()(word_dense)
preds = Dense(K)(word_att)
model = Model(sequence_input, preds)
# lstm_orig = CuDNNLSTM(units= hidden_size, return_sequences=True,
#                                       recurrent_initializer='glorot_uniform')(embedded_sequences)
                                                               
# lstm, forward_h, backward_h = Bidirectional(CuDNNLSTM(units= hidden_size, return_sequences=True,
#                                       recurrent_initializer='glorot_uniform'))(lstm_orig)

# state_h = Concatenate()([forward_h, backward_h])

# context_vector, attention_weights = attention(lstm, state_h)

# output = keras.layers.Dense(1, activation='sigmoid')(context_vector)

# model = keras.Model(inputs=sequence_input, outputs=output)

# model.add(Dense(K, activation='softmax'))

############################ Non Model Layers #################################################


# summarize layers
# print(model.summary())
    # SGD optimizer with momentum
optimizer = Adam(lr=learning_rate, decay=1e-6)
    # compile model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    # training
model.fit(train_data_matrix, train_data_label, epochs=total_epoch, batch_size=batch_size)
    # testing
train_score = model.evaluate(train_data_matrix, train_data_label, batch_size=batch_size)
print('Training Loss: {}\n Training Accuracy: {}\n'.format(train_score[0], train_score[1]))
valid_score = model.evaluate(valid_data_matrix, valid_data_label, batch_size=batch_size)
print('Validation Loss: {}\n Validation Accuracy: {}\n'.format(valid_score[0], valid_score[1]))
    # predicting
test_pre = model.predict(test_data_matrix, batch_size=batch_size).argmax(axis=-1) + 1
sub_df = pd.DataFrame()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [66]:
df = pd.read_csv('data/train.csv')
lengths = []
tokens = []
length = 0 
for text in df['text']:
    for word in nltk.word_tokenize(text):
        word = word.lower()
        if word not in stop_words and not word.isnumeric():
            tokens.append(word)
            length = length + 1 
    lengths.append(length)
    length = 0 
                          

In [70]:
lengths_dict = {'lengths': lengths}
lengths_df = pd.DataFrame(data=lengths_dict)
lengths_df.describe()

,lengths
count,100000.000000
mean,58.093860
std,53.804495
min,0.000000
25%,24.000000
50%,41.000000
75%,74.000000
max,769.000000


In [14]:
x = plt.hist(lengths)


NameError: name 'lengths' is not defined

In [76]:
x

(array([7.6413e+04, 1.7955e+04, 3.9240e+03, 1.0690e+03, 3.7900e+02,
        1.8400e+02, 7.0000e+01, 4.0000e+00, 1.0000e+00, 1.0000e+00]),
 array([  0. ,  76.9, 153.8, 230.7, 307.6, 384.5, 461.4, 538.3, 615.2,
        692.1, 769. ]),
 <a list of 10 Patch objects>)

In [42]:
x = pd.DataFrame({"health": 100}, in

In [44]:
x['health']

0    100
Name: health, dtype: int64

,health
0,100
